In [228]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', -1)
outfile =  "datos.html"
datafile = "../gente.txt"


# Voy a sanitizar la tabla, por si aparece algun registro con menos campos o texto

questioncols = ['asistencia', 'cumple_horarios', 'buen_trato', 'clase_organizada', 'claridad', 'fomenta_participacion', 'panorama_amplio', 'acepta_critica', 'responde_mails']
usecols = ['doc', 'mat'] + questioncols + ['comentarios']

df_raw = pd.read_csv(datafile, usecols=usecols, header=0)
for index in questioncols:
    df_raw[index] = pd.to_numeric(df_raw[index], errors='coerce') # Furzo estos campos a numerico
    
df = data=df_raw.dropna() # Elimino campos con errores
df = df[df['doc']!="A Designar"] # Elimino el docente "A Designar"
#materias = df.mat.unique()
df.head()

,doc,mat,asistencia,cumple_horarios,buen_trato,clase_organizada,claridad,fomenta_participacion,panorama_amplio,acepta_critica,responde_mails,comentarios
0,Garcia,6109,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,VW5hIGdlbmlhIHRvdGFsLCByZXNwb25kZSBhIHRvZG9zIGxvcyBlbWFpbHMsIHkgc2llbXByZSBlc3RhIHByZXNlbnRlLiA=
16,Santamartina,8711,3.0,4.0,5.0,1.0,1.0,1.0,5.0,2.0,4.0,TWUgZG9ybcOtIGRlc2RlIHF1ZSBtZSBhbm90ZSBlbiBlbCBzaXUg
17,Sande,8711,5.0,4.0,4.0,3.0,3.0,1.0,3.0,5.0,4.0,Q2hldG8=
33,Echarri,6201,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,3.0,VW4gcG9jbyB2YWdvLCBwZXJvIGxvIG1lam9yIGRlIGZpdWJh
37,Echarri,6201,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,3.0,VW4gcG9jbyB2YWdvLCBwZXJvIGxvIG1lam9yIGRlIGZpdWJh


# Calificación del docente

A fin de poder ordenar los docentes, para que los "mejores" aparezcan primero, es necesario obtener una métrica.
Voy a utilizar una norma 2 pero con coeficientes de ponderación dados consignados en la siguiente tabla.

In [229]:
pesos = {
    'asistencia': 1,
    'cumple_horarios': 1,
    'clase_organizada': .7,
    'claridad': .7,
    'buen_trato': 0.5,
    'acepta_critica': 0.5,
    'fomenta_participacion': 0.5,
    'responde_mails': 0.5,
    'panorama_amplio': 0.5
}
features = list(pesos.keys())
w = np.array(list(pesos.values()))
wn = w/np.sum(w)
puntajes = np.sqrt(np.square(df[features].values) @ wn)

# Tabla de comparación

Tomando la media de cada feature, analizando para docente y materia por separado

In [230]:
# Para un docente particular
docentes = list(df['doc'].unique()) # pss... lista de docentes
df[df['doc']=='Acero'][features].mean()

asistencia               5.000000
cumple_horarios          5.000000
clase_organizada         4.285714
claridad                 5.000000
buen_trato               4.142857
acepta_critica           4.428571
fomenta_participacion    4.000000
responde_mails           3.000000
panorama_amplio          4.857143
dtype: float64

In [231]:
# Para una materia en particular
puntajes = df[df['mat']==6103].groupby('doc')[features].mean()
puntajes.head()

,asistencia,cumple_horarios,clase_organizada,claridad,buen_trato,acepta_critica,fomenta_participacion,responde_mails,panorama_amplio
doc,,,,,,,,,
Acero,5.0,5.0,4.75,5.0,4.5,4.75,4.25,3.0,5.0
Boggi,2.0,3.0,3.00,4.0,4.0,4.00,4.00,2.0,4.0
Cristi,5.0,5.0,5.00,5.0,5.0,5.00,5.00,5.0,5.0
De Rossi,3.0,3.0,4.00,4.0,4.0,2.00,4.00,1.0,3.0
Liberczuk,1.0,1.0,1.00,1.0,1.0,1.00,1.00,1.0,1.0


## Índice múltiple
Tendría que repetir el procedimiento para cada materia, pero si uso un indice múltiple (de la forma (materia,docente))
me ahorro todo ese trabajo.

In [232]:
comentarios = df.groupby(['mat','doc'])['comentarios'].apply(list).to_frame("comentarios")
counts = df.groupby(['mat','doc']).size().to_frame(name='respuestas')
grouped = df.groupby(['mat','doc'])[features].mean().join(counts).join(comentarios)

grouped

asistencia  cumple_horarios  clase_organizada  \
mat  doc                                                                
6103 Acero              5.000000    5.0              4.750000           
     Boggi              2.000000    3.0              3.000000           
     Cristi             5.000000    5.0              5.000000           
     De Rossi           3.000000    3.0              4.000000           
     Liberczuk          1.000000    1.0              1.000000           
     Lopez              5.000000    5.0              5.000000           
     Maulhardt          5.000000    4.8              5.000000           
     Muszkats           5.000000    5.0              5.000000           
     Palacios           5.000000    5.0              5.000000           
     Puebla             2.000000    2.0              3.000000           
     Ramirez Oyhanarte  3.000000    2.5              2.500000           
     Remesar            4.000000    4.0              2.000000           
     Rosito             1.000000    2.0              3.000000           
     Seminara           5.000000    5.0              5.000000           
     Sirne              5.000000    4.0              5.000000           
     Sosa               5.000000    5.0              5.000000           
     Unger              5.000000    5.0              4.000000           
     Vorobioff          5.000000    5.0              3.000000           
     Zitto              4.000000    4.0              4.000000           
6106 Baliña             5.000000    5.0              3.500000           
     Cosatto Ammann     5.000000    5.0              5.000000           
     Garcia Galiñanes   5.000000    5.0              5.000000           
     Gogni              5.000000    5.0              5.000000           
     Kornblit           4.000000    3.0              2.000000           
     Villaverde         5.000000    5.0              5.000000           
6107 Canga              1.000000    3.0              2.000000           
6108 Arrejoria          5.000000    5.0              5.000000           
     Boggi              1.000000    1.0              1.000000           
     Cabana             3.000000    2.0              5.000000           
     Calzon             4.000000    4.0              5.000000           
...                          ...    ...                   ...           
8711 Raiden             5.000000    5.0              5.000000           
     Sande              5.000000    4.0              4.000000           
     Santamartina       3.333333    4.0              3.333333           
8713 Perez              5.000000    4.0              2.000000           
     Sauque             5.000000    5.0              5.000000           
8714 Hutin              3.000000    3.0              1.000000           
     Pardi              1.000000    1.0              1.000000           
     Roit               1.000000    1.0              1.000000           
8917 Calzaretto         5.000000    5.0              5.000000           
     Lasanta            5.000000    1.0              1.000000           
9103 Bursky             5.000000    4.0              3.000000           
     Cosatto Ammann     5.000000    4.0              5.000000           
     Pardieux           3.000000    3.0              2.000000           
9109 Jahn               5.000000    5.0              1.000000           
     Vallejos           5.000000    5.0              3.000000           
9133 Botta              2.000000    1.0              2.000000           
     Carrizo            4.000000    3.0              1.000000           
     Palavecino         4.000000    4.0              1.000000           
9203 Pinchete           5.000000    3.0              1.000000           
9208 Cofone             1.000000    1.0              1.000000           
9501 Costa              5.000000    3.0              3.000000           
     Lanzillotta        4.000000    4.0              3.000000           
     Pere

In [233]:
import base64
import lxml.html

comment = comentarios['comentarios'][5][1]
#res = [ for x in comment]

def b64_to_html_decoding(comment_list):
    text = "[ULSTART]"
    for x in comment_list:
        #text = text + lxml.html.tostring(lxml.html.fromstring(base64.b64decode(x))).decode("ascii").replace("<p>","•").replace("</p>","\n")
        text = text + "[LISTART]" + base64.b64decode(x).decode("utf8")+"[LIEND]"
    return text + "[ULEND]"

#lxml.html.tostring(lxml.html.fromstring(base64.b64decode(comment))).decode("ascii").replace("<p>","<li>").replace("</p>","</li>")

grouped['comentarios'] = grouped['comentarios'].apply(b64_to_html_decoding)
grouped.to_json(orient='index')

'{"[6103,"Acero"]":{"asistencia":5.0,"cumple_horarios":5.0,"clase_organizada":4.75,"claridad":5.0,"buen_trato":4.5,"acepta_critica":4.75,"fomenta_participacion":4.25,"responde_mails":3.0,"panorama_amplio":5.0,"respuestas":4,"comentarios":"[ULSTART][LISTART]A0 es lo m\\u00e1s grande que hay en el mundo[LIEND][LISTART]Acero es uno de los mejores profesores que uno pueda encontrarse, sabe much\\u00edsimo y no da vueltas con los temas, explica los ejercicios clave y ense\\u00f1a a justificar. Se llega m\\u00e1s que preparado para dar el final. El problema que tiene es que va muy r\\u00e1pido y es dif\\u00edcil llevar al d\\u00eda una materia as\\u00ed yendo tan r\\u00e1pido y en cuanto te atrasas, las te\\u00f3ricas cuesta entenderlas. Es un curso recomendado para recursantes. Corrige \\u00e9l los parciales y a las horas ya est\\u00e1n subidas las notas al campus, tambi\\u00e9n responde dudas por mensajes o antes de las clases.[LIEND][LISTART]Excelente profesor aunque anotarse con \\u00e9l

In [238]:
from decimal import Decimal
import datetime

def float_format(x):
    return str(round(Decimal(x),1))

html = grouped.to_html(float_format=float_format)

now = datetime.datetime.now()

html_head = '<!DOCTYPE html>\
<html>\
<head>\
  <meta charset="utf-8" />\
  <meta name="viewport" content="width=device-width, minimum-scale=1, maximum-scale=1" />\
  <meta http-equiv="Expires" content="0">\
  <meta http-equiv="Last-Modified" content="0">\
  <meta http-equiv="Cache-Control" content="no-cache, mustrevalidate">\
  <meta http-equiv="Pragma" content="no-cache">\
  <title>Resultados</title></head><body>'

header = "<h4>Actualizado: " + str(now) +"</h4>"
with open(outfile, mode="w", encoding="utf8") as f:
    f.write(html_head+header+html+'</body></html>')